In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper - I added 'inspect'
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [4]:
 # create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [6]:
 # View all of the classes that automap found
Base.classes.keys()    

['measurement', 'station']

In [7]:
 # Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
 # Create our session (link) from Python to the DB
session = Session(engine)

In [9]:
# Identify the columns for measurement table.
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [10]:
# Take a look at 5 measurement records.
engine.execute('select * from measurement limit 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [11]:
# Identify columns for station table.
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [12]:
# Take a look at 5 station records.
engine.execute('select * from station limit 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

# Exploratory Precipitation Analysis

In [13]:
# Number of dates in the measurement table.
session.query(func.count(Measurement.date)).all()


[(19550)]

In [14]:
 # Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
recent_date

('2017-08-23')

In [15]:
# Starting from the most recent data point in the database. 

date = dt.date(2017, 8, 23)
date

datetime.date(2017, 8, 23)

In [16]:
# Calculate the date one year from the last date in data set.
# start_date = dt.date(2017, 8, 23) - dt.timedelta(days = 365)
# print("Year ago date: ", start_date)

In [26]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Perform a query to retrieve the data and precipitation scores
# year_data = session.query(Measurement.date, Measurement.prcp).\
year_data = session.query(Measurement.date).filter(Measurement.date <= '2017-08-23').\
    filter(Measurement.date >= '2016-08-23').count()
year_data

2230

In [18]:
# # Save the query results as a Pandas DataFrame and set the index to the date column
# year_data = pd.DataFrame(year_data, columns=['date', 'prcp'])
# year_data.set_index('date', inplace = True)
# year_data.plot.bar()
# plt.tight_layout()
# plt.show()

In [19]:

# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data

In [20]:
 # Use Pandas to calcualte the summary statistics for the precipitation data

# Exploratory Station Analysis

In [21]:
# Design a query to calculate the total number stations in the dataset

In [22]:
 # Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

In [23]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

In [24]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

# Close session

In [25]:
 # Close Session
session.close()